# 정적 크롤링 : BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib

## 1. 미니 실습

In [2]:
html = """
<html>
  <body>
    <div id="main" class="content">
      <h1>첫 번째 헤더</h1>
      <h1 class="header">두 번째 헤더</h1>
      <p class="description">설명입니다.</p>
      <a href="https://example1.com" class="link">링크 1</a>
      <a href="https://example2.com" class="link">링크 2</a>
      <a href="https://example3.com" class="external">링크 3</a>
    </div>
    <div id="main2" class="content">
      <div class="items">
        <div>
            <img class="thumbnail" src="/data/picture1.png" />
            <div class="title">타이틀1</div>
            <div class="description">설명1</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture2.png" />
            <div class="title">타이틀2</div>
            <div class="description">설명2</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture3.png" />
            <div class="title test">타이틀3</div>
            <div class="description">설명3</div>
        </div>
      </div>
    </div>
  </body>
</html>
"""

### 1) HTML 파싱 준비

In [3]:
soup = BeautifulSoup(html, "html.parser")
print(soup)


<html>
<body>
<div class="content" id="main">
<h1>첫 번째 헤더</h1>
<h1 class="header">두 번째 헤더</h1>
<p class="description">설명입니다.</p>
<a class="link" href="https://example1.com">링크 1</a>
<a class="link" href="https://example2.com">링크 2</a>
<a class="external" href="https://example3.com">링크 3</a>
</div>
<div class="content" id="main2">
<div class="items">
<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
</div>
</div>
</body>
</html>



### 2) 파싱

In [4]:
# 맨 처음 나오는 class 이름이 title인 div를 추출하라

print(soup.find("div", class_ = "title"))

# class 이름이 title인 div를 모두 추출하라

print(soup.find_all("div", class_ = "title"))


<div class="title">타이틀1</div>
[<div class="title">타이틀1</div>, <div class="title">타이틀2</div>, <div class="title test">타이틀3</div>]


In [5]:
# 미션 : "두 번째 헤더" 를 출력하라

print(soup.find("h1", class_ = "header").text)

# 미션 : "설명3" 을 출력하라

print(soup.find_all("div", class_ = "description")[2].text)

# 미션 : 태그 a를 모두 출력하라

temp_list = soup.find_all("a")
for tem in temp_list :
    print(tem.text)

두 번째 헤더
설명3
링크 1
링크 2
링크 3


In [6]:
# 미션 :
# html에서 image, title, description 열을 가진 데이터 프레임을 만들어라
# 안에 들어갈 데이터는 image의 url, 제목, 설명을 넣어라

image_list = []
title_list = []
description_list = []

temp_image = soup.find_all("img")
for tag in temp_image :
    image_list.append(tag["src"])

temp_title = soup.find_all("div", class_ = "title")
for tag in temp_title :
    title_list.append(tag.text)

temp_description = soup.find_all("div", class_ = "description")
for tag in temp_description :
    description_list.append(tag.text)

sum_data_frame = pd.DataFrame({
    'image' : image_list,
    'title' : title_list,
    'description' : description_list
})
print(sum_data_frame)



# tag = div 이며 class 이름이 items 인 데이터를 추출하라

itmes_html = soup.find("div", class_ = "items")

# items_html 에서 tag = div 만 추출하라

div_outputs = itmes_html.find_all("div", recursive = False)
div_html = ""
for div in div_outputs :
    div_html += f"{str(div)}\n"

print(div_html)

# 각 요소를 추출하라

data_list = []

for div in div_outputs :
    temp_dict = {}
    temp_dict["image"] = (div.find("img")["src"])
    temp_dict["title"] = (div.find("div", class_ = "title").text)
    temp_dict["description"] = (div.find("div", class_ = "description").text)
    data_list.append(temp_dict)

test_data_frame = pd.DataFrame(data_list)
test_data_frame

                image title description
0  /data/picture1.png  타이틀1         설명1
1  /data/picture2.png  타이틀2         설명2
2  /data/picture3.png  타이틀3         설명3
<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>



,image,title,description
0,/data/picture1.png,타이틀1,설명1
1,/data/picture2.png,타이틀2,설명2
2,/data/picture3.png,타이틀3,설명3


In [7]:
# 링크 1을 출력하라

print(soup.select_one('div > a').text)

# 링크 2를 출력하라

print(soup.select_one('div > a:nth-child(5)').text)

링크 1
링크 2


In [8]:
# class가 items 인 것을 출력해주세요

print(soup.select('div.items'))

#

[<div class="items">
<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
</div>]


## 2. 교보문고 실습

```
https://search.kyobobook.co.kr/search?keyword=%EC%B0%BD%EC%97%85&target=sam&gbCode=SAM&page=1
```

### 1) HTML 요청하기

In [9]:
base_url = "https://search.kyobobook.co.kr/search"
keyword = "창업"
target = "kyobo"
gbCode = "SAM"
page = 1

url = f"{base_url}?keyword={keyword}&target={target}&gbCode={gbCode}&page={page}"

params = {
    "keyword" : keyword,
    "target" : target,
    "gbCode" : gbCode,
    "page" : page
}

headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(base_url, params = params, headers = headers)
response


<Response [200]>

In [10]:
print(response.text)

<!DOCTYPE html>
<html lang="ko"
    data-view="ink"
    data-service="search"
>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    
        
            
            <meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1">
        
        
        

    
        <meta name="title" content="창업 검색결과 | 교보문고">
        <meta property="og:title" content="창업 검색결과 | 교보문고">
    

    
        <meta name="description" content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다.">
        <meta property="og:description" content="꿈을 키우는 세상 교보문고는 온오프라인을 통틀어 대한민국 최고의 도서쇼핑몰이며 전자책, 음반, 기프트, 문화서비스까지 제공하는 종합문화기업입니다.">
    

    <meta property="og:type" content="website">
    <meta property="og:image" content="https://contents.kyobobook.co.kr/resources/fo/images/common/ink/img_logo_kyobo@2x.png">
    <meta property="og:url" content="http://mobile.kyobobook.co.kr">
    
    



<meta 

### 2) HTML 파싱 준비

In [11]:
html = response.text
soup = BeautifulSoup(html)

### 3) 파싱

In [12]:
# tag = ul, class = prod_list

prod_list = soup.find("ul", class_ = "prod_list")

# prod list 에서 tag = li

prod_item = soup.find_all("li", class_ = "prod_item")
print(len(prod_item))

20


In [37]:
book_info_list = []
for i in range(0, len(prod_item)) :
    book_info_dict_temp = {}
    book_info_dict_temp["book_name"] = prod_item[i].select("div.auto_overflow_wrap.prod_name_group > div.auto_overflow_contents > div.auto_overflow_inner > a > span")[-1].text
    book_info_dict_temp["book_writer"] = prod_item[i].select_one("div.auto_overflow_wrap.prod_author_group > div.auto_overflow_contents > div > a").text
    book_info_dict_temp["price"] = str(prod_item[i].select_one("div.prod_price > span.price > span.val").text) + "원"
    book_info_dict_temp["grade"] = prod_item[i].select_one("span.review_klover_text.font_size_xxs").text

    book_info_list.append(book_info_dict_temp)

book_data_frame = pd.DataFrame(book_info_list)
book_data_frame

,book_name,book_writer,price,grade
0,미니멀리스트 창업가,사힐 라빈지아,"17,100원",0.0
1,"창업과 세금절세 및 부가가치세, 종합소득세 세무신고 직접하기",이진규,"31,500원",0.0
2,브랜드 창업 마스터,이종구,"20,700원",9.95
3,양조장 할 생각은 없었는데요,박정범,"12,060원",10.0
4,초보 사업자의 물음표 세무형이 답하다,이의유,"18,000원",10.0
5,린 스타트업,에릭 리스,"16,200원",9.53
6,라멘집 창업한 회계사의 실전 회계학개론,이시도 류,"18,000원",10.0
7,창업 1년 차 김 사장은 어떻게 투자 유치에 성공했을까,이응진,"16,200원",10.0
8,"로컬의 힘, 지역경제를 바꾸다",하상용,"17,100원",10.0
9,당신은 사업가입니까,캐럴 로스,"22,500원",10.0
